In [1]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#load camera
cap = cv2.VideoCapture(0)

#determine the size of the capture
cap_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
cap_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

#intial dot is random about the middle of the screen ensure that the dot is about the middle of the screen since the cascade classifier only detects full frontal faces
dot_position = (np.random.randint(0.2*cap_width, 0.7*cap_width), np.random.randint(0.2*cap_height, 0.7*cap_height))

#initialize face position at the middle of the screen
face_position = (cap_width // 2, cap_height // 2)

#initialize starting score
score = 0

In [2]:
while True:
    boolean, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #the parameters are tweaked to improve accuracy such as the minimum and maximum face size 
    faces = face_cascade.detectMultiScale(gray, 2, 3, minSize=(100, 100), maxSize=(1000, 1000))

    # draw the red circle using the dot_position as the coordinates
    cv2.circle(frame, dot_position, 15, (0, 0, 255), -1)

    # for loop for the detected faces
    for (x, y, w, h) in faces:
        
        # draw green rectangle with a cross on the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        cv2.line(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        cv2.line(frame, (x + w, y), (x, y + h), (0, 255, 0), 1)

        # the position of the face is detected and the velocity is detected by subtracting last face position to the current face position
        # the face detection can work without accounting for the velocity but is placed there to improve the jittering
        face_x = x + w // 2
        face_y = y + h // 2
        face_position = (face_x, face_y)
        face_velocity = (face_x - face_position[0], face_y - face_position[1])

        # the approximation of the nose coordinates is the sum of the position and the velocity 
        nose_x = face_position[0] + face_velocity[0]
        nose_y = face_position[1] + face_velocity[1]

        # if fucntion to detect if the coordinates of the nose is near the position of the red dot, if true the dot will change positions and the score will increase by one
        if (nose_x - dot_position[0]) ** 2 + (nose_y - dot_position[1]) ** 2 < 300 :
            dot_position = (np.random.randint(0.2*frame.shape[1], 0.7*frame.shape[1]), np.random.randint(0.2*frame.shape[0], 0.7*frame.shape[0]))
            score +=1
        
        # the score tally 
        cv2.putText(frame, 'Score: ' + str(score), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow('Face Detection Game', frame)

    # close the game
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()